In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import gc


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
# load the data
df_original = pd.read_csv('/kaggle/input/vu-dmt-assigment-2-2023/training_set_VU_DM.csv',nrows=100000)

In [ ]:
# sample 10000 instances of the data
df = df_original#.sample(n = 50000)
print("the shape of the data ",df.shape)
df.head()

In [ ]:
# Check for missing values
df.isnull().sum(axis = 0)

In [ ]:
import missingno as msno
# visualize the missing values
msno.bar(df)

In [ ]:
df.describe()

In [ ]:
# Check the distribution of the booking_bool variable
print(f"Check the distribution of the booking_bool variable \n{df['booking_bool'].value_counts()}\n")

# Check the distribution of the click_bool variable
print(f"Check the distribution of the click_bool variable \n{df['click_bool'].value_counts()}\n")

# Check the distribution of the visitor_location_country_id 
print(f"Check the distribution of the visitor_location_country_id \n{df['visitor_location_country_id'].value_counts().head()}\n")

# Check the distribution of the visitor_hist_starrating 
print(f"Check the distribution of the visitor_hist_starrating \n{df['visitor_hist_starrating'].value_counts().head()}\n")

# average visitor_hist_starrating for each srch_id
print(f"average visitor_hist_starrating for each srch_id \n{df.groupby('srch_id')['visitor_hist_starrating'].mean().dropna().head()}\n")

# average visitor_location_country_id for each srch_id
print(f"average visitor_location_country_id for each srch_id \n{df.groupby('srch_id')['visitor_location_country_id'].mean().dropna().head()}\n")


In [ ]:
# Assuming you have already loaded the data into a DataFrame called 'df'
df['date_time'] = pd.to_datetime(df['date_time'])  # Convert 'date_time' column to datetime format
df_sorted = df.sort_values('date_time')  # Sort the DataFrame based on 'date_time'

# Plotting the sorted data
plt.figure(figsize=(10,5))
plt.plot(df_sorted['date_time'], df_sorted['price_usd'])
plt.xlabel('Date')
plt.ylabel('Price')
plt.show()

In [ ]:
# # Visualize the distribution of numerical features
# num_features = ['booking_bool','visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_starrating', 
#                 'prop_review_score', 'prop_location_score1', 'prop_location_score2', 
#                 'prop_log_historical_price', 'position', 'price_usd',
#                 'srch_query_affinity_score', 'orig_destination_distance',
#                 'comp1_rate', 'comp1_inv', 'comp1_rate_percent_diff', 
#                 'comp2_rate', 'comp2_inv', 'comp2_rate_percent_diff', 
#                 'comp3_rate', 'comp3_inv', 'comp3_rate_percent_diff', 'comp4_rate',
#                 'comp4_inv', 'comp4_rate_percent_diff', 'comp5_rate', 'comp5_inv', 
#                 'comp5_rate_percent_diff', 'comp6_rate', 'comp6_inv', 
#                 'comp6_rate_percent_diff', 'comp7_rate', 'comp7_inv', 
#                 'comp7_rate_percent_diff', 'comp8_rate', 'comp8_inv',
#                 'comp8_rate_percent_diff']

# fig, axes = plt.subplots(9, 4, figsize=(20, 30))
# axes = axes.flatten()

# for i, feature in enumerate(num_features):
#     sns.histplot(df[feature], kde=False, ax=axes[i])
#     axes[i].set_title(feature)

# plt.tight_layout()
# plt.show()

In [ ]:
# Visualize the correlation between features
# Create a correlation matrix
corr = df.corr()
# Set up the plot
fig, ax = plt.subplots(figsize=(12, 10))
# Create a heatmap of the correlation matrix
sns.heatmap(corr, cmap='coolwarm', ax=ax)
# Show the plot
plt.show()

In [ ]:
# drop the features are not in the test set
new_df =df.drop(['position','click_bool','booking_bool','gross_bookings_usd'],axis=1)

# Define a list of columns to fill missing values for
columns_to_fill = ['visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_review_score', 'prop_location_score2',
                   'srch_query_affinity_score', 'orig_destination_distance', 'comp1_rate', 'comp1_inv', 
                   'comp1_rate_percent_diff', 'comp2_rate', 'comp2_inv', 'comp2_rate_percent_diff', 'comp3_rate', 
                   'comp3_inv', 'comp3_rate_percent_diff', 'comp4_rate', 'comp4_inv', 'comp4_rate_percent_diff', 
                   'comp5_rate', 'comp5_inv', 'comp5_rate_percent_diff', 'comp6_rate', 'comp6_inv',
                   'comp6_rate_percent_diff', 'comp7_rate', 'comp7_inv', 'comp7_rate_percent_diff',
                   'comp8_rate', 'comp8_inv', 'comp8_rate_percent_diff']


# Loop through each column and fill missing values with the first quartile of the respective country
for column in columns_to_fill:
    # Group the data by country and calculate the first quartile of the column
    first_quartile_by_country = round(new_df.groupby("prop_country_id")[column].quantile(0.25))
     # Fill missing values in the column with the first quartile of the respective country
    new_df[column].fillna(new_df["prop_country_id"].map(first_quartile_by_country), inplace=True)
    
    

In [ ]:
# check if still some missing values
new_df.isnull().sum()

In [ ]:
# drop the features still have a lot of missing values
new_df=new_df.drop(['comp6_rate','comp6_inv','comp6_rate_percent_diff'],axis=1)
# fill the missing values with forward filling method 
new_df = new_df.fillna(method='ffill')
new_df.isnull().sum()

In [ ]:
# perform principal component analysis to reduce the dimensionality
from sklearn.decomposition import PCA
def perform_pca(df, pca_dict, dimensions):
    for feature_set_name, feature_set in pca_dict.items():
        # Select the columns corresponding to the numeric features
        data = df[feature_set]        
        # Drop old features from the original DataFrame
        df = df.drop(columns=feature_set)        
        # Perform PCA
        pca = PCA(n_components=dimensions)
        transformed_data = pca.fit_transform(data)        
        # Create a new DataFrame with the transformed data and use the dictionary element name as column name
        new_df = pd.DataFrame(transformed_data, columns=[f"{feature_set_name}_{i+1}" for i in range(dimensions)])        
        # Concatenate the new DataFrame with the original DataFrame
        df = pd.concat([df, new_df], axis=1)    
    return df

pca_dict={'comp_rate_pca':['comp1_rate','comp2_rate','comp3_rate',
                              'comp4_rate','comp5_rate','comp7_rate','comp8_rate'],
         'comp_inv_pca':['comp1_inv','comp2_inv','comp3_inv',
                              'comp4_inv','comp5_inv','comp7_inv','comp8_inv'],
         'comp_rate_percent_diff_pca':['comp1_rate_percent_diff','comp2_rate_percent_diff','comp3_rate_percent_diff',
                              'comp4_rate_percent_diff','comp5_rate_percent_diff','comp7_rate_percent_diff','comp8_rate_percent_diff']}
new_df1= perform_pca(new_df,pca_dict,1)
new_df1.head()

In [ ]:
data = new_df1

# data = new_df
data = data.drop(['srch_id', 'site_id', 'visitor_hist_starrating', 'visitor_hist_adr_usd'], axis=1)

# convert date_time to year, month, day, and hour
data['date_time'] = pd.to_datetime(data['date_time'])
data['year'] = data['date_time'].dt.year
data['month'] = data['date_time'].dt.month
data['day'] = data['date_time'].dt.day
data['hour'] = data['date_time'].dt.hour
data = data.drop('date_time', axis=1)

# # fill missing values with mean
# data['prop_review_score'] = data['prop_review_score'].fillna(data['prop_review_score'].mean())

# create new features
data['log_price_diff'] = data['prop_log_historical_price'] - np.log(data['price_usd']+1)
data['price_rank'] = data.groupby('prop_id')['price_usd'].rank(method='dense')

data['star_rank'] = data.groupby('visitor_location_country_id')['prop_starrating'].rank(method='dense', ascending=False)

# Calculate average price by country and star rating
avg_price = data.groupby(['prop_country_id', 'prop_starrating'])['price_usd'].transform('mean')
# Calculate price difference and Add price difference as a new feature
data['price_diff'] = data['price_usd'] - avg_price


# To combine the prop_location_score1 and prop_location_score2 columns to create a new feature that captures 
# the overall location score of the hotel, we can simply add these two columns
data['location_score'] = data['prop_location_score1'] + data['prop_location_score2']

print(data.shape)
data.head()

In [ ]:
x = data['location_score'].unique()
x.sort()
print(len(x))
print(x)

In [ ]:
####################################################

In [ ]:
# for balance the target classes 
from sklearn.utils import resample

# Separate majority and minority classes
df_majority = df[df.booking_bool==0]
df_minority = df[df.booking_bool==1]

# Downsample the majority class to match the minority class
df_majority_downsampled = resample(df_majority, 
                                   replace=False,   # sample without replacement
                                   n_samples=len(df_minority),  # match number in minority class
                                   random_state=42)  # reproducible results

# Combine the downsampled majority class with the minority class
df_balanced = pd.concat([df_majority_downsampled, df_minority])

# Check the class distribution
print(df_balanced.booking_bool.value_counts())

In [ ]:
df_balanced.to_csv('expedia_balanced.csv', index=False)


In [ ]:
from sklearn.neighbors import NearestNeighbors

# create the user-hotel matrix
user_hotel_matrix = df.pivot_table(values='booking_bool', index='visitor_location_country_id', columns='prop_id', fill_value=0)
# create a nearest neighbors model
model = NearestNeighbors(metric='cosine', algorithm='brute')

# fit the model to the user-hotel matrix
model.fit(user_hotel_matrix)

# get the nearest neighbors for a given user
user_index = 1
distances, indices = model.kneighbors(user_hotel_matrix.iloc[user_index,:].values.reshape(1, -1), n_neighbors=5)

# use the nearest neighbors to make predictions for the user
c = user_hotel_matrix.iloc[indices[0],:].mean(axis=0)

print(c)

In [ ]:
user_hotel_matrix.shape

In [ ]:
import pandas as pd
from scipy.stats import pearsonr


# Get a list of all binary variables in the dataset
binary_vars = [col for col in df.columns if df[col].dtype == 'int64']

# Get a list of all continuous variables in the dataset
continuous_vars = [col for col in df.columns if df[col].dtype == 'float64']

# Calculate the point-biserial correlation coefficient for each pair of variables
results = []
# for binary_var in binary_vars:
for continuous_var in continuous_vars:
    r_pb, p_value = pearsonr(df['booking_boll'], df[continuous_var])
    results.append((binary_var, continuous_var, r_pb, p_value))

# Convert the results to a pandas DataFrame and save to a CSV file
df_results = pd.DataFrame(results, columns=['binary_var', 'continuous_var', 'r_pb', 'p_value'])
# df_results.to_csv('expedia_point_biserial_correlations.csv', index=False)

# Print the results for the top 10 correlations
top_correlations = df_results.sort_values(by='r_pb', ascending=False).head(10)
print(top_correlations)
